# Requirements

In [ ]:
!python -m pip install "pymongo[srv]" groq langchain-mongodb langchain pymongo bs4 openai tiktoken gradio requests lxml unstructured langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (set

In [ ]:
!pip install -U sentence-transformers
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-cohere cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.8 MB/s eta 0:00:00


In [ ]:
import pymongo
import requests
from transformers import AutoModel
from sentence_transformers import SentenceTransformer
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import json
import cohere
from groq import Groq
from langchain_groq import ChatGroq
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
import gradio as gr
import gradio.themes.base as Base
import os
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_cohere import ChatCohere

# Searching in vectordatabse

In [ ]:
client = MongoClient('Your cluster goeshere')
db = client.sample_mflix
collection = db.movies


hf_token = 'yourhftoken'
embedding_url = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
def generate_embedding(text:str) -> list[float]:
  response = requests.post(
      embedding_url,
      headers = {"Authorization": f"Bearer {hf_token}"},
      json = {'inputs':text},)

  if response.status_code !=200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")
  return response.json()

for doc in collection.find({'plot':{"$exists": True}}).limit(60):
    doc['plot_embeddig_hf'] = generate_embedding(doc['plot'])
    collection.replace_one({'_id': doc['_id']}, doc)


In [ ]:
query = "imaginary characters from outer space at war"

In [ ]:
results = collection.aggregate([
    {"$vectorSearch":{
        "queryVector":generate_embedding(query),
        "path":'plot_embeddig_hf',
        "numCandidates":100,
        "limit":6,
        "index":"PlotSemanticSearch"
    }}
]);

In [ ]:
for document in results:
    print(f'Movie Name: {document["title"]},\nMovie Plot: {document["plot"]}\n')

Movie Name: Four Sons,
Movie Plot: A family saga in which three of a Bavarian widow's sons go to war for Germany and the fourth goes to America, Germany's eventual opponent.

Movie Name: The Strong Man,
Movie Plot: A meek Belgian soldier (Harry Langdon) fighting in World War I receives penpal letters and a photo from "Mary Brown", an American girl he has never met. He becomes infatuated with her by ...

Movie Name: The Four Horsemen of the Apocalypse,
Movie Plot: An extended family split up in France and Germany find themselves on opposing sides of the battlefield during World War I.

Movie Name: In the Land of the Head Hunters,
Movie Plot: Original advertising for the film describes it as a drama of primitive life on the shores of the North Pacific...

Movie Name: Wild Oranges,
Movie Plot: Millie Stope lives with her grandfather on a remote island. Her grandfather fled there for political reasons. But they're not alone. An escaped prisoner, Nicholas, is terrorizing them, and ...

Movi

# Atlas vector search combined with llm chating

In [ ]:
!git clone https://github.com/beaucarnes/vector-search-tutorial.git

Cloning into 'vector-search-tutorial'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 96 (delta 3), reused 3 (delta 3), pack-reused 84
Receiving objects: 100% (96/96), 220.86 KiB | 8.49 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!mkdir Sampledata

### Uploading search data

In [ ]:
MONGO_URI='yourcluster'
client = MongoClient(MONGO_URI)
dbName = "langchain_demo"
collectionName = "collection_of_text_blobs"
collection = client[dbName][collectionName]
# Directory containing your .txt files
directory_path = '/content/Sampledata'

In [ ]:
for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
        # Read the text file
        with open(os.path.join(directory_path, filename), 'r') as file:
            text_content = file.read()

        # Generate embeddings for the text
        embeddings = model.encode(text_content).tolist()

        # Create a JSON object
        document_json = {
            'text': text_content,
            'embeddings': embeddings
        }

        # Insert the JSON object into the MongoDB collection
        collection.insert_one(document_json)

print("Data and embeddings have been successfully inserted into the MongoDB collection.")

Data and embeddings have been successfully inserted into the MongoDB collection.


## Retriving data from mongodb

In [ ]:
MONGO_URI='ypurcluster'
client = MongoClient(MONGO_URI)
dbName = "langchain_demo"
collectionName = "collection_of_text_blobs"
collection = client[dbName][collectionName]
vectorstore = MongoDBAtlasVectorSearch(collection, model)

In [ ]:
# Function to query and retrieve data using the "default" index
def retrive_infromation(query):
    # Perform a search using the "default" index
    results = collection.find(
        {"$text": {"$search": query}},
        {"score": {"$meta": "textScore"}}
    ).sort([("score", {"$meta": "textScore"})])

    # Iterate over the results and print the text and embeddings
    for doc in results:
        print("Text:", doc['text'])
        print("Embeddings:", doc['embeddings'])

# Example usage of the function
retrive_infromation('What is Boundaries?')

Text: Alex: Hey Jamie, I've been reading about psychology recently, and I came across the concept of boundaries. Can you tell me more about it?

Jamie: Of course, Alex! In psychology, boundaries refer to the limits we set with other people. They indicate what's acceptable behavior and what isn't. They're crucial for maintaining a healthy relationship and a sense of individuality.

Alex: That sounds interesting. But how do I know where to draw these boundaries?

Jamie: It's a personal decision and can vary from person to person. It depends on your comfort level, your values, and your past experiences. It's about understanding what you're comfortable with and communicating that effectively to others.

Alex: I see. But isn't it rude to tell someone that they're crossing your boundaries?

Jamie: Not at all, Alex. It's important to communicate your boundaries clearly and assertively. It's not about controlling others but about taking care of your own mental and emotional health.

Alex: That

In [ ]:
co = cohere.Client('yourcoherecode')

# Function to retrieve embeddings and generate output using the Cohere model
def retrieve_and_generate(query):
    # Perform a search using the "default" index
    results = collection.find(
        {"$text": {"$search": query}},
        {"score": {"$meta": "textScore"}}
    ).sort([("score", {"$meta": "textScore"})])

    # Create a list to store the embeddings and texts
    embeddings_list = []
    texts_list = []

    # Iterate over the results and add the embeddings and texts to the lists
    for doc in results:
        embeddings_list.append(doc['embeddings'])
        texts_list.append(doc['text'])

    if embeddings_list and texts_list:
        # Truncate the embeddings list to save space
        embeddings_list = embeddings_list[:10]  # Adjust this value as needed

        # Convert the embeddings list to a string format
        embeddings_str = json.dumps(embeddings_list)

        # Join the texts into a single string
        texts_str = ' '.join(texts_list)

        # Create the prompt template
        message = "Could you tell me what Alex said in bullet points?" f"{texts_str} {embeddings_str}"

        # Use the Cohere model with the prompt template
        cohere_output = co.chat(message=message)

        # Split the output text into sentences and print each sentence on a new line
        for sentence in cohere_output.text.split('\n'):
            print(sentence)
    else:
        print("No matching documents found.")

# Example usage of the function
retrieve_and_generate('What is Boundaries?')


- Alex asks Jamie to explain the concept of boundaries in psychology.
- Alex wants to know how to determine where to set their own boundaries.
- Alex questions if it is rude to tell someone they are crossing a boundary.
- Alex asks what to do if someone doesn't respect their boundaries.
- Alex concludes that setting boundaries is about self-care and self-respect.


### Using Cohere RAG to search and print about boundaries

In [ ]:
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_community.retrievers import CohereRagRetriever
from langchain_core.documents import Document

# Set your Cohere API key
os.environ["CommandRapi"] = getpass.getpass()

# Initialize the Cohere RAG retriever
rag = CohereRagRetriever(llm=ChatCohere(cohere_api_key='yourcoherekey'))

# Function to retrieve embeddings and generate output using the Cohere model
def retrieve_and_generate(query):
    # Use the Cohere RAG retriever to find relevant documents
    docs = rag.invoke(query)

    # Iterate over the documents and print each one
    for doc in docs:
        print(doc.metadata)
        print("\n\n" + doc.page_content)
        print("\n\n" + "-" * 30 + "\n\n")

# Example usage of the function
retrieve_and_generate('What is Boundaries?')


··········


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `CohereRagRetriever` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRagRetriever`.
  warn_deprecated(


{'id': 'web-search_4', 'snippet': "Boundaries: Definition, Examples & How To Set Them\n\nBy Jaya Roy, M.A., ASW \u200b\n\nReviewed by Tchiki Davis, M.A., Ph.D.\n\nBoundaries are important for all types of relationships. Learn how to set them here.\n\n*This page may include affiliate links; that means we earn from qualifying purchases of products.\n\nWhat Are Boundaries? (A Definition)\n\nWhen you think about the word boundary, you might have an image of a fence or of a border come to your mind. The fence might be used to keep two things separate from each other or to provide protection from another entity. \n\nWhile this idea of a boundary is clear cut, psychological boundaries—or the standard by which we want people to treat us—are harder to identify. Psychological boundaries are also less fixed and change depending on circumstances such as the situation, one’s values, and cognitive, physical, or emotional capacity. \u200bBefore we get started, we thought you might be interested in ta

### Integrating to use this as flask app

In [ ]:
from flask import Flask, render_template, request
import cohere
import json

app = Flask(__name__)
co = cohere.Client('yourocherekey')

# Your existing retrieve_and_generate function here
def retrieve_and_generate(query):
    # Perform a search using the "default" index
    results = collection.find(
        {"$text": {"$search": query}}, {"score": {"$meta": "textScore"}}
    ).sort([("score", {"$meta": "textScore"})])

    # Create a list to store the embeddings and texts
    embeddings_list = []
    texts_list = []

    # Iterate over the results and add the embeddings and texts to the lists
    for doc in results:
        embeddings_list.append(doc['embeddings'])
        texts_list.append(doc['text'])

    if embeddings_list and texts_list:
        # Truncate the embeddings list to save space
        embeddings_list = embeddings_list[:10]  # Adjust this value as needed

        # Convert the embeddings list to a string format
        embeddings_str = json.dumps(embeddings_list)

        # Join the texts into a single string
        texts_str = ' '.join(texts_list)

        # Create the prompt template
        message = "Could you tell me what Alex said in bullet points?" f"{texts_str} {embeddings_str}"

        # Use the Cohere model with the prompt template
        cohere_output = co.chat(message=message)

        # Split the output text into sentences and return as a list
        return cohere_output.text.split('\n')
    else:
        return ["No matching documents found."]
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        query = request.form['query']
        output = retrieve_and_generate(query)
        return render_template('index.html', output=output)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
